In [1]:
!pip install requests beautifulsoup4

In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time

def get_date_list(start_date_str, end_date_str):
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    if start_date.weekday() != 6:
        raise ValueError("start_date must be a Sunday (weekday=6)")
    if end_date.weekday() != 5:
        raise ValueError("end_date must be a Saturday (weekday=5)")
    date_list = []

    while start_date <= end_date:
        date_list.append(start_date.strftime("%Y-%m-%d"))
        start_date += timedelta(days=7)  # 주간차트 얻기 위함.

    return date_list

def scrape_billboard(date):
    url = f"https://www.billboard.com/charts/hot-100/{date}"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    chart_items = soup.select("div.o-chart-results-list-row-container")

    weekly_results = []

    for item in chart_items[:100]:  # 최대 100위까지 시도
        try:
            ranking = int(item.select_one("span.c-label.a-font-primary-bold-l").text.strip())
            title = item.select_one("h3.c-title").text.strip()
            singer = item.select_one("span.c-label.a-no-trucate.a-font-primary-s").text.strip()

            weekly_results.append({
                "basedt": date,
                "music_rank": ranking,
                "music_name": title,
                "artist_name": singer
            })

        except (AttributeError,ValueError):
            continue  # 일부 항목 누락 시 무시

    return weekly_results

def crawl_billboard_range(start_date, end_date, delay_sec=3):
    all_results = []
    status_report = []
    date_list = get_date_list(start_date, end_date)

    print(f"크롤링 날짜 리스트: {date_list}")
    print(f"총 {len(date_list)}개의 일자를 크롤링합니다...\n")

    for date in date_list:
        print(f"{date} → 크롤링 시작...")
        try:
            results = scrape_billboard(date)
            count = len(results)

            if count > 0:
                print(f"{date}: {count}곡 수집 성공")
                status_report.append((date, "성공", count))
            else:
                print(f"{date}: 곡 수집 실패 또는 차트 없음")
                status_report.append((date, "실패", 0))

            all_results.extend(results)

        except Exception as e:
            print(f"{date}: 에러 발생 → {e}")
            status_report.append((date, "에러", 0))

        time.sleep(delay_sec)

    print("\n 날짜별 크롤링 요약:")
    for date, status, count in status_report:
        print(f" - {date} | {status} | {count}곡")

    return all_results


# 사용 예시
start_date = "2025-05-04"
end_date = "2025-05-31"


data = crawl_billboard_range(start_date, end_date)



크롤링 날짜 리스트: ['2025-05-04', '2025-05-11', '2025-05-18', '2025-05-25']
총 4개의 일자를 크롤링합니다...

2025-05-04 → 크롤링 시작...
2025-05-04: 100곡 수집 성공
2025-05-11 → 크롤링 시작...
2025-05-11: 100곡 수집 성공
2025-05-18 → 크롤링 시작...
2025-05-18: 100곡 수집 성공
2025-05-25 → 크롤링 시작...
2025-05-25: 100곡 수집 성공

 날짜별 크롤링 요약:
 - 2025-05-04 | 성공 | 100곡
 - 2025-05-11 | 성공 | 100곡
 - 2025-05-18 | 성공 | 100곡
 - 2025-05-25 | 성공 | 100곡
